In [ ]:
!pip install Bio

     |████████████████████████████████| 102kB 4.8MB/s 
     |████████████████████████████████| 2.3MB 8.6MB/s 


In [ ]:
!pip install pybedtools

     |████████████████████████████████| 12.5MB 237kB/s 
     |████████████████████████████████| 9.9MB 22.6MB/s 
  Created wheel for pybedtools: filename=pybedtools-0.8.2-cp37-cp37m-linux_x86_64.whl size=13617693 sha256=859bba01b58fc540fd33c1a9730ec3a3a811866e87ed8379bffc3b1448659ef4
  Stored in directory: /root/.cache/pip/wheels/ab/ae/8d/efdbbacea1f52e55602720575328b97fed787bd12723d25d5b
Successfully built pybedtools


In [ ]:
import os
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import Entrez

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/Annotation')

# Genes - ORFs

In [ ]:
# BLASTP hittable for ORFs (identified using Artemis)
a = pd.read_csv('HitTable.csv')

In [ ]:
a2 = a.Gene.unique()

In [ ]:
columns = a.columns
new_a = pd.DataFrame(columns=columns)

for gene in a2:
    temp_df = a[a['Gene'] == gene]
    max_identity = temp_df['identity'].max()
    new_a = new_a.append(temp_df[temp_df['identity'] == max_identity])



In [ ]:
new_a.reset_index(drop=True, inplace=True)

In [ ]:
new_a = new_a.reindex(columns = new_a.columns.tolist() + ['start','end'])

In [ ]:
#Genes coordinates - parsing of ORF list from Artemis
coord = SeqIO.parse(open('amino_acids.fasta'),'fasta')

In [ ]:
import re

In [ ]:
regex = r"undefined product (\d+):(\d+)"

i=0
for seq_record in coord:
#     print(seq_record.description)
#     re_match = regex.match(seq_record.description)
#     print(re_match.group(1))
    matches = re.finditer(regex, seq_record.description)
    for matchNum, match in enumerate(matches, start=1):
        new_a.loc[new_a.index[i],'start'] = match.group(1)
        new_a.loc[new_a.index[i],'end'] = match.group(2)
    i+=1

In [ ]:
min(new_a.identity)

In [ ]:
new_a.head(52)

# Genes - Intergenic

In [ ]:
# BLASTP hittable for intergenic regions (identified using Atremis)
b = pd.read_csv('HitTable-intergene.csv')

In [ ]:
b2 = b.Gene.unique()

In [ ]:
columns = b.columns
new_b = pd.DataFrame(columns=columns)

for gene in b2:
    temp_df = b[b['Gene'] == gene]
    max_identity = temp_df['identity'].max()
    new_b = new_b.append(temp_df[temp_df['identity'] == max_identity])

new_b.head()

,Gene,Hit,identity,alignmentlength,mismatches,gapopens,qstart,qend,sstart,send,evalue,bitscore,positives
0,PORF00001,ETY10707.1,97.619,42,1,0,1,42,8,49,3.060000e-19,86.3,97.62
1,PORF00015,AUW38949.1,96.667,30,1,0,1,30,1,30,8.640000e-12,63.5,100.00
11,PORF00017,EVU16135.1,100.000,29,0,0,42,70,1,29,9.350000e-10,58.5,100.00
12,PORF00042,WP_005454126.1,100.000,126,0,0,16,141,1,126,4.840000e-81,246.0,100.00


In [ ]:
new_b = new_b.reindex(columns = new_b.columns.tolist() + ['start','end'])

In [ ]:
#BLASTP hittable for Genes in intergenic regions (coordinates fetched from alignments)
c = pd.read_csv('HitTable-intergene2.csv')

In [ ]:
c2 = c.Gene.unique()
columns = c.columns
new_c = pd.DataFrame(columns=columns)

for gene in c2:
    temp_df = c[c['Gene'] == gene]
    max_identity = temp_df['identity'].max()
    new_c = new_c.append(temp_df[temp_df['identity'] == max_identity])

In [ ]:
new_c = new_c.reindex(columns = new_c.columns.tolist() + ['start','end'])

In [ ]:
#Genes coordinates - parsing of file from Artemis
coord_c = SeqIO.parse(open('intergenic-only with matches.fasta'),'fasta')

In [ ]:
import re

In [ ]:
regex = r"undefined product (\d+):(\d+)"

i=0
for seq_record in coord_c:
    matches = re.finditer(regex, seq_record.description)
    for matchNum, match in enumerate(matches, start=1):
        new_c.loc[new_c.index[i],'start'] = match.group(1)
        new_c.loc[new_c.index[i],'end'] = match.group(2)
    i+=1

In [ ]:
new_c.head()

In [ ]:
min(new_c.identity)

# All genes and functions

In [ ]:
Genes = pd.concat([new_a, new_c])

In [ ]:
Genes.loc[:, 'start'] = pd.to_numeric(Genes.loc[:, 'start'])

In [ ]:
Genes.sort_values(by='start', inplace=True)

In [ ]:
Genes.index = range(len(Genes.index))

In [ ]:
Genes = Genes.reindex(columns = Genes.columns.tolist() + ['organism'] + ['prodct'] + ['intergene'])

In [ ]:
for gene in Genes.Hit:
    handle = Entrez.efetch(db='protein', rettype="gb", retmode="text", id=gene)
    for seq_record in SeqIO.parse(handle, "gb"):
        Genes.prodct[Genes.Hit == gene] = seq_record.description[:70]
        Genes.organism[Genes.Hit == gene] = seq_record.annotations["source"]
    handle.close()

/usr/local/lib/python3.7/dist-packages/Bio/Entrez/__init__.py:670: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be se

In [ ]:
#Add protein function identified using Pfam/InterPro/Plast
Genes.prodct[Genes.Gene == 'ORF00009'] = 'TPA protein'
Genes.prodct[Genes.Gene == 'ORF00016c'] = 'N-acetyltransferase domain-containing protein'
Genes.prodct[Genes.Gene == 'PORF00015'] = 'TPA protein'
Genes.prodct[Genes.Gene == 'PORF00017'] = 'heterocyst differentiation protein HetZ'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [ ]:
#Cut product name after [ sign
for i in range(54):
    Genes.loc[Genes.index[i],'prodct'] = Genes.loc[Genes.index[i],'prodct'].split('[', 1)[0]

# Overlap

In [ ]:
#Identify intergentic regions
for i in range(54):
    Genes.loc[Genes.index[i],'intergene'] = int(Genes.loc[Genes.index[i+1],'start']) - int(Genes.loc[Genes.index[i],'end'])

In [ ]:
#Check for ORF overlap over 50 nt
Genes.Gene[Genes.intergene < -50]

Series([], Name: Gene, dtype: object)

In [ ]:
Genes.head(10)

# Comparison with RAST and Prokka

In [ ]:
RastGenes = pd.read_csv('RASTresults.txt', delimiter="\t")

In [ ]:
from pybedtools import BedTool

In [ ]:
test = BedTool.from_dataframe(Genes)

In [ ]:
test2 = BedTool.from_dataframe(RastGenes)

In [ ]:
result = test.jaccard(test2, f=0.5, r=False)

In [ ]:
#IoU


In [ ]:
RastGenes = RastGenes.reindex(columns = RastGenes.columns.tolist() + ['organism'] + ['prodct'] + ['intergene'])

In [ ]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [ ]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))
list1 = ['dog', 'cat', 'cat', 'rat']
list2 = ['dog', 'cat', 'mouse']
jaccard_similarity(list1, list2)

In [ ]:
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist, jaccard


In [ ]:
res = pdist(df[['category1','category2','category3']], 'jaccard')


In [ ]:
df = pd.DataFrame(Genes.start, Genes.end, RastGenes.start, RastGenes.stop)

In [ ]:
res = pdist(df[[Genes.start, Genes.end], [RastGenes.start, RastGenes.stop]], 'jaccard')

# Results - GenBank file

In [ ]:
# Save as GenBank file
record = SeqRecord(sequence_object,
                   id='123456789', # random accession number
                   name='Example',
                   description='An example GenBank file generated by BioPython')

# Add annotation
feature = SeqFeature(FeatureLocation(start=3, end=12), type='misc_feature')
record.features.append(feature)

output_file = open('example.gb', 'w')
SeqIO.write(record, output_file, 'genbank')

In [ ]:
SeqIO.write(record, output_file, 'genbank')